## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Cockburn Town,TC,21.4612,-71.1419,75.22,65,0,11.63,clear sky,2021-03-15 03:30:54
1,1,Barrow,US,71.2906,-156.7887,-5.80,77,1,11.50,snow,2021-03-15 03:32:50
2,2,Hilo,US,19.7297,-155.0900,71.60,83,90,4.61,light rain,2021-03-15 03:19:47
3,3,Sao Filipe,CV,14.8961,-24.4956,70.68,73,0,4.07,clear sky,2021-03-15 03:34:29
4,4,Port Hardy,CA,50.6996,-127.4199,41.00,80,40,8.05,scattered clouds,2021-03-15 03:34:30


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Cockburn Town,TC,21.4612,-71.1419,75.22,65,0,11.63,clear sky,2021-03-15 03:30:54
2,2,Hilo,US,19.7297,-155.0900,71.60,83,90,4.61,light rain,2021-03-15 03:19:47
3,3,Sao Filipe,CV,14.8961,-24.4956,70.68,73,0,4.07,clear sky,2021-03-15 03:34:29
6,6,Vaini,TO,-21.2000,-175.2000,75.20,94,75,3.44,broken clouds,2021-03-15 03:24:56
7,7,Ribeira Grande,PT,38.5167,-28.7000,60.39,92,34,6.93,scattered clouds,2021-03-15 03:34:30
11,11,Hithadhoo,MV,-0.6000,73.0833,84.24,63,96,6.11,overcast clouds,2021-03-15 03:21:16
13,13,Popondetta,PG,-8.7537,148.2534,88.59,59,78,6.29,broken clouds,2021-03-15 03:34:32
14,14,West Bay,KY,19.3667,-81.4167,78.01,83,20,4.61,few clouds,2021-03-15 03:34:32
15,15,Kontagora,NG,10.3999,5.4695,77.88,27,82,5.19,broken clouds,2021-03-15 03:34:33
17,17,Wahiawa,US,21.5028,-158.0236,75.20,78,20,5.75,light rain,2021-03-15 03:33:06


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                305
City                   305
Country                298
Lat                    305
Lng                    305
Max Temp               305
Humidity               305
Cloudiness             305
Wind Speed             305
Current Description    305
Date                   305
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                298
City                   298
Country                298
Lat                    298
Lng                    298
Max Temp               298
Humidity               298
Cloudiness             298
Wind Speed             298
Current Description    298
Date                   298
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cockburn Town,TC,75.22,clear sky,21.4612,-71.1419,
2,Hilo,US,71.60,light rain,19.7297,-155.0900,
3,Sao Filipe,CV,70.68,clear sky,14.8961,-24.4956,
6,Vaini,TO,75.20,broken clouds,-21.2000,-175.2000,
7,Ribeira Grande,PT,60.39,scattered clouds,38.5167,-28.7000,
11,Hithadhoo,MV,84.24,overcast clouds,-0.6000,73.0833,
13,Popondetta,PG,88.59,broken clouds,-8.7537,148.2534,
14,West Bay,KY,78.01,few clouds,19.3667,-81.4167,
15,Kontagora,NG,77.88,broken clouds,10.3999,5.4695,
17,Wahiawa,US,75.20,light rain,21.5028,-158.0236,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, hotel in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = hotel["Lat"]
    lng = hotel["Lng"]
    params["location"]= f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   298
Country                298
Max Temp               298
Current Description    298
Lat                    298
Lng                    298
Hotel Name             298
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current Description}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_lay
              er)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))